In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from casestudy.update import auto_update

In [36]:
ROOTPATH='/Users/spindicate/Documents/programming/zooscraper/'

In [42]:
from datetime import datetime as dt
note = '\Note: All testset and many of the historical datasets have been removed to reduce the repo size.'
from decouple import config
import nbformat as nbf
from nbconvert import MarkdownExporter
from nbconvert.writers import FilesWriter

README_FILE = config('SEE19PATH') + 'README.ipynb'
README_NAME = config('SEE19PATH') + 'README'

nb = nbf.read(README_FILE, as_version=4)
update_date = dt.now().strftime('%B %d, %Y')
leadcell = {
    'cell_type': 'markdown',
   'metadata': {},
   'source': """# see19\n\n**An aggregation dataset and interface for visualizing 
       and analyzing Coronavirus Disease 2019 aka COVID19 
       aka C19**\n\n*Dataset Last Updated {}*{}
   """.format(update_date, note)
}
nb.cells.insert(0, leadcell)
del nb.cells[1]

# For some reason Node object is held as JSON "in memory"
# so must convert back to NotebookNode
# https://github.com/jupyter/nbconvert/issues/85
nb = nbf.v4.to_notebook(nb)
nbf.write(nb, README_FILE)

readme = nbf.read(README_FILE, as_version=4)
exporter = MarkdownExporter()
(body, resources) = exporter.from_notebook_node(readme)

write_file = FilesWriter()
write_file.write(
    output=body,
    resources=resources,
    notebook_name=README_NAME
)

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_load.v0+json']) is not able to be represented.
  mimetypes=output.keys())
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_exec.v0+json']) is not able to be represented.
  mimetypes=output.keys())


'/Users/spindicate/Documents/programming/zooscraper/casestudy/see19//README.md'

In [4]:
auto_update()

Running get_italy...
Running get_braz...
Running get_US...
Running create_rest...
Running update_austests...
Running update_cadtests...
Running update_ustests...
Running update_resttests...
Running update_strindex...
Running update_gmobi...


/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


Aruba AW
Cote d'Ivoire 
Cape Verde CV
Laos LA
North Macedonia MK
Réunion RE
Tajikistan TJ
Running update_amobi...
Instantiating Chrome WebDriver...
Running update_msmts...
Exception('no data is available within your requested subset. Request returned no data.')
Running update_pollutants...
Running make_baseframe...
initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE
Running test_region_consistency...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Running test_notnas...
Exception(' ...Failing Regions: [407]')
Running test_duplicate_dates...
Running test_duplicate_days...
Running test_negative_days...
Running make_baseframe...
initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfi

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_load.v0+json']) is not able to be represented.
  mimetypes=output.keys())
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/javascript', 'application/vnd.bokehjs_exec.v0+json']) is not able to be represented.
  mimetypes=output.keys())
